# IBM Applied Data Science Capstone
## Part 4: SQL-Based Exploratory Data Analysis

**Objective:** Perform SQL queries on the dataset for structured analysis

**Author:** Son Nguyen

---


In [ ]:
import pandas as pd
import pandasql as psql
import warnings
warnings.filterwarnings('ignore')

# Load data
df = pd.read_csv('../data/automotive_sales.csv')
df['Date'] = pd.to_datetime(df[['Year', 'Month']].assign(Day=1))

print("Dataset loaded successfully!")
print(f"Shape: {df.shape}")


## Query 1: Total Sales by Vehicle Type


In [ ]:
q1 = """
SELECT 
    Vehicle_Type,
    COUNT(*) as Count,
    SUM(Sales) as Total_Sales,
    AVG(Sales) as Avg_Sales,
    SUM(Revenue) as Total_Revenue
FROM df
GROUP BY Vehicle_Type
ORDER BY Total_Sales DESC
"""

result1 = psql.sqldf(q1, locals())
print(result1)
